In [2]:
import logging
import pickle
from sklearn import metrics
from sklearn.metrics import f1_score
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.preprocessing.text import one_hot
from keras.optimizers import Adam
from tqdm import tnrange, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.utils import class_weight

In [37]:
MODEL_PATH = "../models/"

In [4]:
DATA_PATH = "../data/"
X_train = pickle.load(open(DATA_PATH + "X_train.p", "rb"))
X_dev = pickle.load(open(DATA_PATH + "X_dev.p", "rb"))
y_train = pickle.load(open(DATA_PATH + "y_train.p", "rb"))
y_dev = pickle.load(open(DATA_PATH + "y_dev.p", "rb"))

In [9]:
y_train.shape

(111699, 6)

In [6]:
train_text = X_train['comment_text']

In [7]:
tokenizer = Tokenizer(num_words=18400)
tokenizer.fit_on_texts(train_text)
sequences = tokenizer.texts_to_sequences(train_text)

In [14]:
word_index = tokenizer.word_index
embeddings_index = {}
f = open('../../embeddings/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [19]:
print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_layer = Embedding(embedding_matrix.shape[0],
                            300,
                            weights=[embedding_matrix],
                            input_length=200, trainable = False)

Found 400000 word vectors.


In [25]:
max_length = 200

x_train_texts = tokenizer.texts_to_sequences(X_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_dev_texts = tokenizer.texts_to_sequences(X_dev['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')

In [26]:
current_y_train = y_train[:, 0]
current_y_dev = y_dev[:, 0]

In [27]:
class_weight = class_weight.compute_class_weight('balanced', np.unique(current_y_train), current_y_train)
class_weight = {i:x for i, x in enumerate(list(class_weight))}

In [28]:
class_weight

{0: 0.5530365294542862, 1: 5.213732262882749}

In [30]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm = keras.layers.LSTM(10, return_sequences=True)(embedded_sequences)
lstm2 = keras.layers.LSTM(2)(lstm)
x = Dense(1, activation='sigmoid')(lstm2)

model = Model(sequence_input, x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          50649300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 10)           12440     
_________________________________________________________________
lstm_2 (LSTM)                (None, 2)                 104       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 50,661,847
Trainable params: 12,547
Non-trainable params: 50,649,300
_________________________________________________________________


In [ ]:
model.fit(x_train_texts, current_y_train, validation_data=(x_dev_texts, current_y_dev),
          epochs=20, batch_size=20, class_weight = class_weight)

Train on 111699 samples, validate on 47872 samples
Epoch 1/20
111699/111699 [==============================] - 4258s 38ms/step - loss: 0.6891 - acc: 0.3722 - val_loss: 0.6480 - val_acc: 0.9051
Epoch 2/20
111699/111699 [==============================] - 4263s 38ms/step - loss: 0.6888 - acc: 0.3695 - val_loss: 0.6624 - val_acc: 0.9049
Epoch 3/20
111699/111699 [==============================] - 4300s 38ms/step - loss: 0.6888 - acc: 0.4685 - val_loss: 0.7210 - val_acc: 0.1522
Epoch 4/20
111699/111699 [==============================] - 4316s 39ms/step - loss: 0.6892 - acc: 0.3195 - val_loss: 0.7087 - val_acc: 0.1156
Epoch 5/20
111699/111699 [==============================] - 4284s 38ms/step - loss: 0.6486 - acc: 0.5926 - val_loss: 0.5926 - val_acc: 0.6833
Epoch 6/20
111699/111699 [==============================] - 4284s 38ms/step - loss: 0.5008 - acc: 0.7824 - val_loss: 0.5290 - val_acc: 0.7548
Epoch 7/20
111699/111699 [==============================] - 4169s 37ms/step - loss: 0.4543 - acc:

In [38]:
model_json = model.to_json()
with open(MODEL_PATH + "keras_lstm_classification_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(MODEL_PATH + "keras_lstm_classification_model.h5")
print("Saved model to disk")

Saved model to disk


In [35]:
model.evaluate(x_dev_texts, current_y_dev)

47872/47872 [==============================] - 217s 5ms/step


[0.2957064880725534, 0.8744359959893048]